In [3]:
# Import necessary modules
import os
import requests
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv  # To load environment variables from the .env file

In [4]:
# Load environment variables from .env
load_dotenv()

True

In [5]:
# Read the API key from the environment variables
ALPHAVANTAGE_KEY = os.getenv('ALPHAVANTAGE_KEY')
if ALPHAVANTAGE_KEY is None:
    raise Exception("ALPHAVANTAGE_KEY not set in the .env file.")

In [6]:
def get_daily_prices(symbol):
    """
    Fetches daily time series data for the provided symbol from AlphaVantage API.
    
    Parameters:
        symbol (str): The stock symbol to fetch data for (e.g., 'AAPL').
        
    Returns:
        pd.DataFrame: DataFrame containing the time series data with dates and prices.
    """
    url = "https://www.alphavantage.co/query"
    params = {
        'function': 'TIME_SERIES_DAILY',
        'symbol': symbol,
        'outputsize': 'compact',  # Returns the latest 100 data points
        'apikey': ALPHAVANTAGE_KEY
    }
    response = requests.get(url, params=params)
    response.raise_for_status()
    data = response.json()
    
    # Check if there's an error message
    if "Error Message" in data:
        raise Exception(f"API Error: {data['Error Message']}")
    
    # Check if we're being rate limited
    if "Note" in data:
        raise Exception(f"API Note: {data['Note']}")
    
    # The key for daily data is "Time Series (Daily)"
    time_series = data.get("Time Series (Daily)")
    if not time_series:
        raise Exception(f"No time series data returned from API for symbol: {symbol}")
    
    # Convert the dictionary to a DataFrame
    df = pd.DataFrame.from_dict(time_series, orient='index')
    
    # Clean up column names by removing the prefix numbers
    df.columns = ['open', 'high', 'low', 'close', 'volume']
    
    # Convert string values to numeric
    for col in ['open', 'high', 'low', 'close']:
        df[col] = pd.to_numeric(df[col])
    df['volume'] = pd.to_numeric(df['volume'])
    
    # Make sure the index is datetime
    df.index = pd.to_datetime(df.index)
    
    # Sort index in ascending order (oldest to newest)
    df = df.sort_index()
    
    return df

# Try the API call again
symbol = "AAPL"
try:
    df = get_daily_prices(symbol)
    print("Sample data for", symbol)
    print(df.head())
except Exception as e:
    print(f"Error: {e}")

Sample data for AAPL
              open    high      low   close    volume
2024-11-11  225.00  225.70  221.500  224.23  42005602
2024-11-12  224.55  225.59  223.355  224.23  40398299
2024-11-13  224.01  226.65  222.760  225.12  48566217
2024-11-14  225.02  228.87  225.000  228.22  44923941
2024-11-15  226.40  226.92  224.270  225.00  47923696


In [7]:
# Example usage: Fetch data for a sample symbol (e.g., AAPL)
symbol = 'AAPL'
df = get_daily_prices(symbol)
print("Sample data for", symbol)
print(df.head())

Sample data for AAPL
              open    high      low   close    volume
2024-11-11  225.00  225.70  221.500  224.23  42005602
2024-11-12  224.55  225.59  223.355  224.23  40398299
2024-11-13  224.01  226.65  222.760  225.12  48566217
2024-11-14  225.02  228.87  225.000  228.22  44923941
2024-11-15  226.40  226.92  224.270  225.00  47923696


In [8]:
# Establish a database connection using SQLAlchemy and credentials from the .env file
DB_USER = os.getenv('DB_USER')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_HOST = os.getenv('DB_HOST')
DB_PORT = os.getenv('DB_PORT')
DB_NAME = os.getenv('DB_NAME')

if None in (DB_USER, DB_PASSWORD, DB_HOST, DB_PORT, DB_NAME):
    raise Exception("Database credentials are not fully set in the .env file.")

In [14]:
engine = create_engine(f'mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}')

In [16]:
# Write the DataFrame to a MySQL table named 'raw_alpha_vantage_prices'
# if_exists='replace' ensures the table is replaced every time you run the notebook (adjust as needed)
df.to_sql(name='raw_alpha_vantage_prices', con=engine, if_exists='replace', index=False)

print("Data loaded successfully into 'raw_alpha_vantage_prices' table.")

Data loaded successfully into 'raw_alpha_vantage_prices' table.
